In [0]:
import io
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
url='https://raw.githubusercontent.com/scarface961/Bank-Marketting/master/bank-full.csv'
data = pd.read_csv(url,delimiter=';')
#data1 = pd.read_csv("D:/Data Mining and Concepts Learning/Project/bank/bank.csv")

In [3]:
data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [0]:
data=pd.get_dummies(data, columns=["job","marital","education","default","housing","loan","contact","month","poutcome"])

labelencoder = LabelEncoder()
data['y'] = labelencoder.fit_transform(data['y'])

In [5]:
data

,age,balance,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,2143,5,261,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,44,29,5,151,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,33,2,5,76,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,47,1506,5,92,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,33,1,5,198,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,825,17,977,3,-1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
45207,71,1729,17,456,2,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
45208,72,5715,17,1127,5,184,3,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
45209,57,668,17,508,4,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1


In [0]:
def epoch (history):
  # Plot training & validation accuracy values
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

In [0]:
def percentage_split_NN(percentage_split,X,Y):
  x_train,x_test,y_train,y_test = train_test_split(X,Y,stratify=Y,test_size = percentage_split,random_state=0)

  model= create_NN_model()
  history = model.fit(x_train, y_train,validation_data=(x_test,y_test), epochs=150, batch_size=100)
  pred = model.predict_classes(x_test)
  tn,fp,fn,tp = confusion_matrix(y_test,pred).ravel()
  print("Percentage Split")
  Confusion_matrix(tn,fp,fn,tp)
  #epoch(history)
  

In [0]:
def crossvalidate_NN(folds,X,Y):
  kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=0)
  cvscores = []
  tn=0
  fp=0
  fn=0
  tp=0
  for train, test in kfold.split(X, Y):
    model= create_NN_model()
    history = model.fit(X[train], Y[train],validation_data=(X[test],Y[test]),epochs=150, batch_size=100)
    pred = model.predict_classes(X[test])
    #evaluate the model
    tn1, fp1, fn1, tp1 = confusion_matrix(Y[test],pred).ravel()
    tn = tn+tn1
    fp = fp+fp1
    fn = fn+fn1
    tp = tp+tp1

    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
  print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    
  tn=tn/folds
  fp=fp/folds
  fn=fn/folds
  tp=tp/folds
  print("Cross-validation")
  Confusion_matrix(tn,fp,fn,tp)


In [0]:
def percentage_split_SVM(percentage_split,X,Y):

  x_train,x_test,y_train,y_test = train_test_split(X,Y,stratify=Y,test_size = percentage_split,random_state=0)
  clf = svm.SVC(kernel="linear")
  clf.fit(x_train, y_train)
  predict_svm = clf.predict(x_test)
  Confusion_matrix(y_test,predict_svm)


In [0]:
def crossvalidate_SVM(folds,X,Y):
  kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=0)
  cvscores = []
  tn=0
  fp=0
  fn=0
  tp=0
  for train, test in kfold.split(X, Y):
    clf = svm.SVC(kernel="linear")
    clf.fit(X[train], Y[train])
    predict_svm = clf.predict(X[test])
    tn1, fp1, fn1, tp1 = confusion_matrix(Y[test],predict_svm).ravel()
    tn = tn+tn1
    fp = fp+fp1
    fn = fn+fn1
    tp = tp+tp1


    print("Acc: "+repr(accuracy_score(Y[test],predict_svm)))

  tn=tn/folds
  fp=fp/folds
  fn=fn/folds
  tp=tp/folds
  print("Cross-validation")
  Confusion_matrix(tn,fp,fn,tp)


In [0]:
def Confusion_matrix(tn, fp, fn, tp):
  #tn, fp, fn, tp = confusion_matrix(y_test,pred).ravel()
  print("Confusion Matrix")
  print("   yes     no   <<-- Classified as   ")
  # print("   %.2f    %.2f  yes" %(tp,fn))
  # print("   %.2f    %.2f  no" %(fp,tn))
  print("   "+repr(int(tp))+"    "+repr(int(fn))+"        yes")
  print("   "+repr(int(fp))+"    "+repr(int(tn))+"        no")
  accuracy = float((tp+tn)/(tp+tn+fp+fn))
  specificity = float(tn / (tn+fp))
  sensitivity = float(tp / (tp+fn))
  
  if (tp+fp)>=0:
    precision = float(tp / (tp+fp))
    f_score = (2*precision*sensitivity)/(precision+sensitivity) 
  else:
    precision = 0
    f_score = "?"
 
  #print(classification_report(y_test,predictions))
  print("Accuracy: "+repr(accuracy))
  print("Sensitivity: "+repr(sensitivity))
  print("Specificity: "+repr(specificity))
  print("Precision: "+repr(precision))
  print("f_score: "+repr(f_score))

In [0]:
def create_NN_model():
  #Keras Multi Layer
  model = Sequential()
  model.add(Dense(5,input_dim=51,activation='relu'))
  model.add(Dense(5, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  #keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
  #sgd = optimizers.SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
  a = optimizers.adam(lr=0.007)
  model.compile(loss='binary_crossentropy', optimizer=a, metrics=['accuracy'])
  return model


In [22]:
# Splitting the data labels and dataset
Y = data.y.values
X= data.drop('y',axis = 1).values
# X = data.iloc[:,0:53]
# Y = data.iloc[:,52:53]
# print(X)
# #Normalize
# from sklearn import preprocessing
# X = preprocessing.scale(X)
# print(Y)
from sklearn import preprocessing
X = preprocessing.scale(X)
X



array([[ 1.60696496,  0.25641925, -1.29847633, ..., -0.20597248,
        -0.185948  ,  0.47251925],
       [ 0.28852927, -0.43789469, -1.29847633, ..., -0.20597248,
        -0.185948  ,  0.47251925],
       [-0.74738448, -0.44676247, -1.29847633, ..., -0.20597248,
        -0.185948  ,  0.47251925],
       ...,
       [ 2.92540065,  1.42959305,  0.14341818, ..., -0.20597248,
         5.37784754, -2.11631591],
       [ 1.51279098, -0.22802402,  0.14341818, ..., -0.20597248,
        -0.185948  ,  0.47251925],
       [-0.37068857,  0.52836436,  0.14341818, ...,  4.85501757,
        -0.185948  , -2.11631591]])

In [50]:
percentage_split_NN(0.25,X,Y)
print()
print()
print()
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

Train on 33908 samples, validate on 11303 samples
Epoch 1/150
33908/33908 [==============================] - 6s 175us/step - loss: 0.2913 - acc: 0.8815 - val_loss: 0.2252 - val_acc: 0.9031
Epoch 2/150
33908/33908 [==============================] - 1s 25us/step - loss: 0.2227 - acc: 0.9028 - val_loss: 0.2181 - val_acc: 0.9033
Epoch 3/150
33908/33908 [==============================] - 1s 25us/step - loss: 0.2180 - acc: 0.9040 - val_loss: 0.2168 - val_acc: 0.9027
Epoch 4/150
33908/33908 [==============================] - 1s 26us/step - loss: 0.2164 - acc: 0.9048 - val_loss: 0.2157 - val_acc: 0.9037
Epoch 5/150
33908/33908 [==============================] - 1s 26us/step - loss: 0.2145 - acc: 0.9045 - val_loss: 0.2140 - val_acc: 0.9036
Epoch 6/150
33908/33908 [==============================] - 1s 25us/step - loss: 0.2133 - acc: 0.9047 - val_loss: 0.2145 - val_acc: 0.9042
Epoch 7/150
33908/33908 [==============================] - 1s 25us/step - loss: 0.2126 - acc: 0.9052 - val_loss: 0.2138 -

In [51]:
crossvalidate_NN(10,X,Y)
print()
print()
print()
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

Train on 40689 samples, validate on 4522 samples
Epoch 1/150
40689/40689 [==============================] - 6s 141us/step - loss: 0.2573 - acc: 0.8895 - val_loss: 0.2255 - val_acc: 0.9027
Epoch 2/150
40689/40689 [==============================] - 1s 21us/step - loss: 0.2171 - acc: 0.9012 - val_loss: 0.2155 - val_acc: 0.9045
Epoch 3/150
40689/40689 [==============================] - 1s 21us/step - loss: 0.2118 - acc: 0.9027 - val_loss: 0.2112 - val_acc: 0.9034
Epoch 4/150
40689/40689 [==============================] - 1s 21us/step - loss: 0.2097 - acc: 0.9035 - val_loss: 0.2111 - val_acc: 0.9067
Epoch 5/150
40689/40689 [==============================] - 1s 22us/step - loss: 0.2078 - acc: 0.9039 - val_loss: 0.2120 - val_acc: 0.9054
Epoch 6/150
40689/40689 [==============================] - 1s 21us/step - loss: 0.2067 - acc: 0.9057 - val_loss: 0.2126 - val_acc: 0.9076
Epoch 7/150
40689/40689 [==============================] - 1s 21us/step - loss: 0.2065 - acc: 0.9049 - val_loss: 0.2096 - 

In [0]:
# clf = svm.SVC(kernel="linear")
# clf.fit(x_train, y_train)
# predict_svm = clf.predict(x_test)

In [0]:
# Confusion_matrix(y_test,predict_svm)

Confusion Matrix
   0     1   << Classified as   
   29    101
   35    966
Accuracy: 0.8797524314765695
Sensitivity: 0.2230769230769231
Specificity: 0.965034965034965


In [0]:
crossvalidate_SVM(10,X,Y)

Acc: 0.8741721854304636
Acc: 0.8761061946902655
Acc: 0.8849557522123894
Acc: 0.8938053097345132
Acc: 0.8915929203539823
Acc: 0.8738938053097345
Acc: 0.8761061946902655
Acc: 0.8938053097345132
Acc: 0.8938053097345132
Acc: 0.8938053097345132
Cross-validation
Confusion Matrix
   0     1   << Classified as   
   15    37
   14    385
Accuracy: 0.8815207780725022
Sensitivity: 0.28982725527831094
Specificity: 0.9627500000000001
